<a href="https://colab.research.google.com/github/tm-ty/colab-notebooks-tm/blob/main/YouTube_Transcriptions_em_Massa_(Batch_Transcriptions)(working)_v4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# v4.1 - Com sistema de identificação de Channel ID + sistema de cache para evitar Download Repetido (FUNCIONANDO / WORKING✅)
**Principais características desta versão:**

Identificação automática do ID do canal: O usuário pode inserir qualquer formato de URL do canal (ID direto, nome de usuário ou URL personalizada com "@").

Sistema de cache: Evita o download repetido de legendas já processadas anteriormente.

Download de legendas: Baixa as legendas de todos os vídeos do canal especificado.

Criação de arquivo ZIP: Compacta todas as legendas em um único arquivo para facilitar o download.

**Como usar:**

1. Cole este código em uma única célula no Google Colab.
2. Substitua `API_YT_BS` pelo nome que você definiu na variável de ambiente do Google Colab da sua API do Youtube. Então ATENÇÃO! Nesse exemplo o nome da variável está como `API_YT_BS`, troque esse nome colocando o mesmo nome que você definir na váriável de ambiente (coloque uma nomenclatura que te ajude a identificar qual API você está usando).
3. Execute a célula.
4. **Quando solicitado, insira a URL do canal do YouTube desejado**.
5. O script processará o canal, baixará as legendas (evitando duplicatas graças ao cache) e criará um arquivo ZIP para download.

Esta versão integrada oferece uma experiência mais amigável para o usuário, lidando automaticamente com diferentes formatos de URL de canal, evitando downloads repetidos e simplificando todo o processo em uma única execução.




In [ ]:
# Importações necessárias
import os
import json
import re
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
import zipfile
from google.colab import files
from google.colab import userdata

# Configuração da API
API_KEY = userdata.get('API_YT_BS')  # Obtém a chave da API das variáveis de ambiente
youtube = build('youtube', 'v3', developerKey=API_KEY)

# Nome do arquivo de cache
CACHE_FILE = 'legendas_cache.json'

def obter_id_canal(url):
    padrao_id = r'(?:https?:\/\/)?(?:www\.)?youtube\.com\/channel\/([a-zA-Z0-9_-]+)'
    padrao_user = r'(?:https?:\/\/)?(?:www\.)?youtube\.com\/user\/([a-zA-Z0-9_-]+)'
    padrao_custom = r'(?:https?:\/\/)?(?:www\.)?youtube\.com\/@([a-zA-Z0-9_-]+)'

    match_id = re.match(padrao_id, url)
    if match_id:
        return match_id.group(1)

    match_user = re.match(padrao_user, url)
    if match_user:
        try:
            response = youtube.channels().list(
                part='id',
                forUsername=match_user.group(1)
            ).execute()
            if 'items' in response:
                return response['items'][0]['id']
        except Exception as e:
            print(f"Erro ao obter ID do canal para usuário: {e}")

    match_custom = re.match(padrao_custom, url)
    if match_custom:
        try:
            response = youtube.search().list(
                part='snippet',
                q=match_custom.group(1),
                type='channel'
            ).execute()
            if 'items' in response:
                return response['items'][0]['snippet']['channelId']
        except Exception as e:
            print(f"Erro ao obter ID do canal para URL personalizada: {e}")

    return None

def carregar_cache():
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, 'r') as f:
            return json.load(f)
    return {}

def salvar_cache(cache):
    with open(CACHE_FILE, 'w') as f:
        json.dump(cache, f)

def obter_videos_do_canal(channel_id):
    videos = []
    request = youtube.search().list(
        part="id,snippet",
        channelId=channel_id,
        type="video",
        maxResults=50
    )

    while request:
        response = request.execute()

        for item in response['items']:
            videos.append({
                'id': item['id']['videoId'],
                'title': item['snippet']['title']
            })

        request = youtube.search().list_next(request, response)

    return videos

def baixar_legendas(video_id, titulo, idioma='pt', cache=None):
    if cache is None:
        cache = {}

    if video_id in cache:
        print(f"Legendas para {titulo} já existem. Pulando...")
        return cache[video_id]

    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[idioma])

        titulo_limpo = ''.join(c for c in titulo if c.isalnum() or c in (' ', '_')).rstrip()
        nome_arquivo = f"{titulo_limpo}_{video_id}_{idioma}.txt"

        with open(nome_arquivo, 'w', encoding='utf-8') as arquivo:
            for entrada in transcript:
                arquivo.write(f"{entrada['text']}\n")

        print(f"Legendas baixadas com sucesso: {nome_arquivo}")
        cache[video_id] = nome_arquivo
        salvar_cache(cache)
        return nome_arquivo
    except Exception as e:
        print(f"Erro ao baixar legendas para {titulo}: {str(e)}")
        return None

def processar_canal(channel_id):
    cache = carregar_cache()
    videos = obter_videos_do_canal(channel_id)
    arquivos_baixados = []
    for video in videos:
        arquivo = baixar_legendas(video['id'], video['title'], cache=cache)
        if arquivo:
            arquivos_baixados.append(arquivo)
    return arquivos_baixados

# Interface do usuário
url_canal = input("Digite a URL do canal do YouTube: ")
id_canal = obter_id_canal(url_canal)

if id_canal:
    print(f"ID do canal: {id_canal}")
    arquivos_baixados = processar_canal(id_canal)

    print(f"Total de {len(arquivos_baixados)} arquivos de legendas processados.")

    # Cria um arquivo zip com todas as legendas
    with zipfile.ZipFile('todas_legendas.zip', 'w') as zipf:
        for arquivo in arquivos_baixados:
            zipf.write(arquivo)

    print(f"Arquivo ZIP criado com {len(arquivos_baixados)} legendas.")
    print("Iniciando o download do arquivo ZIP...")

    # Faz o download do arquivo zip
    files.download('todas_legendas.zip')
else:
    print("Não foi possível obter o ID do canal.")

# v4.2 - Com sistema de identificação de Channel ID + sistema de cache para evitar Download Repetido (FUNCIONANDO / WORKING✅)
implementamos uma versão completa e integrada do código que inclui todas as funcionalidades discutidas anteriormente. Esta versão unificada inclui:

1. Sistema de cache para evitar downloads repetidos
2. Suporte para entrada de URL do canal, nome de usuário com @ ou ID do canal
3. Download automático de legendas para todos os vídeos do canal
4. Criação de um arquivo ZIP com todas as legendas baixadas.

**Para usar este código:**

1. Copie e cole todo o bloco de código em uma única célula no Google Colab.
2. Substitua `API_YT_BS` pelo nome que você definiu na variável de ambiente do Google Colab da sua API do Youtube. Então ATENÇÃO! Nesse exemplo o nome da variável está como `API_YT_BS`, troque esse nome colocando o mesmo nome que você definir na váriável de ambiente (coloque uma nomenclatura que te ajude a identificar qual API você está usando).
3. Execute a célula.
4. **Quando solicitado, insira o URL do canal**, nome de usuário com @ ou ID do canal desejado.

**O script irá:**

- Identificar o canal corretamente
- Baixar as legendas de todos os vídeos do canal (pulando aqueles já baixados recentemente)
- Criar um arquivo ZIP com todas as legendas
- Iniciar o download do arquivo ZIP automaticamente

**Benefícios desta versão:**

- Código mais organizado e fácil de manter
- Funcionalidades integradas para uma experiência de usuário mais suave
- Sistema de cache eficiente para evitar downloads desnecessários
- Suporte para diferentes formatos de entrada do canal

In [7]:
!pip install youtube_transcript_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 13.8 MB/s eta 0:00:00


In [8]:
# Dependências Necessárias para o Ambiente do Google Colab
#!pip install youtube_transcript_api

# Importações necessárias
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
import os
import zipfile
from google.colab import files
from google.colab import userdata
import json
from datetime import datetime, timedelta

# Configuração da API do YouTube
API_KEY = userdata.get('API_YT_BS')  # Obtém a chave da API das variáveis de ambiente
youtube = build('youtube', 'v3', developerKey=API_KEY)

# Funções de cache
def carregar_cache():
    if os.path.exists('cache.json'):
        with open('cache.json', 'r') as f:
            return json.load(f)
    return {}

def salvar_cache(cache):
    with open('cache.json', 'w') as f:
        json.dump(cache, f)

def video_processado_recentemente(cache, video_id, dias_validade=7):
    if video_id in cache:
        data_processamento = datetime.fromisoformat(cache[video_id])
        if datetime.now() - data_processamento < timedelta(days=dias_validade):
            return True
    return False

# Função para obter ID do canal
def obter_id_canal(channel_input):
    if channel_input.startswith(('UC', 'HC')):
        return channel_input  # Já é um ID de canal válido

    if '@' in channel_input:
        channel_input = channel_input.replace('https://www.youtube.com/', '').replace('@', '')

    try:
        request = youtube.search().list(
            part="id",
            q=channel_input,
            type="channel",
            maxResults=1
        )
        response = request.execute()

        if 'items' in response and response['items']:
            return response['items'][0]['id']['channelId']
        else:
            raise Exception("Canal não encontrado")
    except Exception as e:
        print(f"Erro ao obter ID do canal: {str(e)}")
        return None

# Função para obter vídeos do canal
def obter_videos_do_canal(channel_id):
    videos = []
    request = youtube.search().list(
        part="id,snippet",
        channelId=channel_id,
        type="video",
        maxResults=50
    )

    while request:
        response = request.execute()

        for item in response['items']:
            videos.append({
                'id': item['id']['videoId'],
                'title': item['snippet']['title']
            })

        request = youtube.search().list_next(request, response)

    return videos

# Função para baixar legendas
def baixar_legendas(video_id, titulo, idioma='pt', cache=None):
    if cache is None:
        cache = carregar_cache()

    if video_processado_recentemente(cache, video_id):
        print(f"Legendas para {titulo} já foram baixadas recentemente. Pulando...")
        return None

    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[idioma])

        titulo_limpo = ''.join(c for c in titulo if c.isalnum() or c in (' ', '_')).rstrip()
        nome_arquivo = f"{titulo_limpo}_{video_id}_{idioma}.txt"

        with open(nome_arquivo, 'w', encoding='utf-8') as arquivo:
            for entrada in transcript:
                arquivo.write(f"{entrada['text']}\n")

        print(f"Legendas baixadas com sucesso: {nome_arquivo}")

        cache[video_id] = datetime.now().isoformat()
        salvar_cache(cache)

        return nome_arquivo
    except Exception as e:
        print(f"Erro ao baixar legendas para {titulo}: {str(e)}")
        return None

# Função para processar o canal
def processar_canal(channel_id):
    cache = carregar_cache()
    videos = obter_videos_do_canal(channel_id)
    arquivos_baixados = []
    for video in videos:
        arquivo = baixar_legendas(video['id'], video['title'], cache=cache)
        if arquivo:
            arquivos_baixados.append(arquivo)
    return arquivos_baixados

# Função principal
def baixar_legendas_canal(channel_input):
    channel_id = obter_id_canal(channel_input)
    if not channel_id:
        print("Não foi possível obter o ID do canal.")
        return

    print(f"ID do canal: {channel_id}")
    arquivos_baixados = processar_canal(channel_id)
    print(f"Total de {len(arquivos_baixados)} arquivos de legendas criados.")

    # Cria um arquivo zip com todas as legendas
    if arquivos_baixados:
        with zipfile.ZipFile('todas_legendas.zip', 'w') as zipf:
            for arquivo in arquivos_baixados:
                zipf.write(arquivo)
                os.remove(arquivo)  # Remove o arquivo .txt após adicioná-lo ao zip

        print(f"Arquivo ZIP criado com {len(arquivos_baixados)} legendas.")
        print("Iniciando o download do arquivo ZIP...")
        files.download('todas_legendas.zip')
    else:
        print("Nenhum arquivo de legenda foi baixado.")

# Exemplo de uso
if __name__ == "__main__":
    channel_input = input("Digite o URL do canal, nome de usuário com @ ou ID do canal: ")
    baixar_legendas_canal(channel_input)

Digite o URL do canal, nome de usuário com @ ou ID do canal: https://www.youtube.com/@CanalSubido
ID do canal: UC4ovz3ObT9fffGjJjVKFGDQ
Erro ao baixar legendas para Como funcionam os anúncios online: O GUIA DEFINITIVO: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=qncdEl3QYeE! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Legendas baixadas com sucesso: Como estabelecer uma rotina para faturar com tráfego pago em 2025  Live 319_zHOi_4gCr8Q_pt.txt
Erro ao baixar legendas para Como prospectar, precificar e fechar os MELHORES CLIE

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>